## Un'esplorazione del word embedding

Luca Mari, marzo 2024

[i file di questa attività: [embed.ipynb](embed.ipynb), [embedutils.py](embedutils.py), [embedexpl.py](embedexpl.py), [embedtempl.html](embedtempl.html)]

**Obiettivi**: comprendere la logica del "word embedding", il processo con cui una rete neurale artificiale converte espressioni linguistiche semplici (parole appunto, e più correttamente token, da un vocabolario) nei vettori numerici su cui i transformer poi operano.  
**Precompetenze**: basi di Python.

Occorre creare un ambiente di lavoro Python, supponiamo con VSCode:
* installare un interprete Python
* scaricare da https://code.visualstudio.com/download e installare VSCode
* eseguire VSCode e attivare le estensioni per Python e Jupyter
* ancora in VSCode:
    * creare una cartella di lavoro e renderla la cartella corrente:  
    * copiare nella cartella questo notebook e il file `embedutils.py` e aprire il notebook
    * creare un ambiente virtuale locale Python (Select Kernel | Python Environments | Create Python Environment | Venv, e scegliere un interprete Python):  
    * installare i moduli Python richiesti, eseguendo dal terminale:  
        `pip install torch transformers multimethod`

Lo spazio di word embedding può essere esplorato anche interattivamente, mediante un'applicazione web. Per eseguire il visualizzatore interattivo web occorre:
* aver eseguito almeno una volta questo notebook, in modo da aver scaricato il transformer `BERT`
* copiare nella stessa cartella con questo notebook e `embedutils.py` anche i file `embedexpl.py` e `embedtempl.html`
* installare altri moduli Python, ancora eseguendo dal terminale:  
    `pip install flask networkx`
* eseguire il visualizzatore `embedexpl.py`
* una volta avviata l'applicazione, aprire un browser e digitare l'indirizzo:  
    `http://127.0.0.1:5000`

Per prima cosa, importiamo il modulo che contiene le funzioni per consentire un accesso "di alto livello" al modello pre-addestrato che opererà sia come tokenizzatore sia come sistema di embedding, usando in questo caso una versione pre-addestrata e fine tuned, su testi in italiano, di `BERT`, che è un transformer "open" (https://it.wikipedia.org/wiki/BERT) ed eseguibile anche localmente (alla prima esecuzione sarà dunque necessario attendere che il modello sia scaricato dal sito di Hugging Face: è un file di circa 400 MB che viene copiato nella cartella HFdata della propria cartella personale) (non discutiamo qui di come questo modello sia stato addestrato a fare embedding).

Dopo aver caricato il modello, verifichiamo che il processo sia andato a buon fine visualizzando le due informazioni principali:
* il numero di token riconosciuti nel vocabolario del modello (`model.vocab_size`);
* la dimensione del vettore in cui ogni token viene embedded (`model.embedding_dim`).

In [2]:
from embedutils import Model
from pprint import pprint

model = Model('dbmdz/bert-base-italian-xxl-cased', True)

print(f"Il tokenizzatore ha un vocabolario di {model.vocab_size} token che riconosce.")
print(f"Ogni token viene mappato ('embedded') in un vettore di {model.embedding_dim} numeri.")
print(f"La matrice degli embeddings ha perciò dimensione {model.vocab_embeddings.shape}")

Il tokenizzatore ha un vocabolario di 31102 token che riconosce.
Ogni token viene mappato ('embedded') in un vettore di 768 numeri.
La matrice degli embeddings ha perciò dimensione (32102, 768)


Il tokenizzatore mantiene un vocabolario dei token che riconosce, in una tabella in cui a ogni token è associato un identificatore univoco (`id`).

In [3]:
token = "bellezza"
token_id = model.token_to_id(token)
print(f"Dato un token, come '{token}', il tokenizzatore è in grado di trovarne l'identificatore: {token_id}")
print(f"(ai token non presenti nel vocabolario è associato l'identificatore {model.tokenizer.convert_tokens_to_ids(model.tokenizer.unk_token)}).")

Dato un token, come 'bellezza', il tokenizzatore è in grado di trovarne l'identificatore: 6108
(ai token non presenti nel vocabolario è associato l'identificatore 101).


Il modello è stato addestrato a mappare ('embed') ogni token, con il suo identificatore, in un vettore di numeri (c'è da considerare che i transformer, come `BERT`, operano sulla base di un embedding dinamico, in cui il vettore di numeri associato a ogni token dipende anche dal contesto ('embedding posizionale'): qui noi lavoriamo solo con la componente statica del vettore).

In [4]:
embedding = model.token_to_embedding(token)
print(f"Il token '{token}' è mappato in un vettore i cui primi 10 elementi sono {embedding[:10]}")

Il token 'bellezza' è mappato in un vettore i cui primi 10 elementi sono [ 0.03051873  0.01173639 -0.04997671  0.0277972   0.02349026  0.00617846
 -0.04041     0.06573588 -0.02823343  0.02561735]


L'embedding è dunque una funzione dall'insieme dei token riconosciuti, cioè il vocabolario, allo spazio metrico dei vettori a *n* dimensioni. Il modello che realizza tale funzione è addestrato in modo tale da cercare di indurre una struttura metrica sul vocabolario, sulla base del principio che token di significato simile dovrebbero essere associati a vettori vicini. Dato un token, è così possibile elencare i token che gli sono più simili nel vocabolario, cioè quelli che associati a vettori più vicini al vettore del token dato.

In [4]:
pprint(model.most_similar(token, top_n=5, filter=True))

[('bellezze', 0.45),
 ('dolcezza', 0.38),
 ('splendore', 0.38),
 ('estetica', 0.35),
 ('fascino', 0.34)]


L'embedding consente di operare in modo piuttosto sofisticato sul vocabolario.  
Per esempio, dati due termini A e B a ognuno dei quali è stato associato un vettore, v(A) e v(B), se la regola di associazione v(.) è sufficientemente ricca da un punto di vista semantico allora il vettore differenza v(A)-v(B) è associato alla relazione tra A e B. In questo modo diventa possibile operare con relazioni di "proporzionalità semantica", del tipo: data la relazione tra "re" e "uomo", qual è il termine X che è nella stessa relazione con "donna"? Questa domanda è dunque codificata come v("re")-v("uomo)=v(X)-v("donna"), e perciò v(X)=v("re")+v("donna")-v("uomo), in cui "re" e "donna" sono gli "esempi positivi" e "uomo" è l'"esempio negativo".

In [10]:
positive_examples = ["re", "donna"]             # sovrano donna
negative_examples = ["uomo"]
print(model.most_similar(positive_examples, negative_examples, top_n=1, filter=True))

[('regina', 0.31)]


Alcuni altri esempi analoghi.

In [11]:
positive_examples = ["Roma", "Francia"]        # capitale della Francia
negative_examples = ["Italia"]
print(model.most_similar(positive_examples, negative_examples, top_n=1, filter=True))

[('Parigi', 0.48)]


In [12]:
positive_examples = ["Italia", "Catalogna"]     # stato a cui appartiene la Catalogna
negative_examples = ["Lombardia"]
print(model.most_similar(positive_examples, negative_examples, top_n=1, filter=True))

[('Spagna', 0.39)]


In [13]:
positive_examples = ["padre", "figlia"]         # genitore femmina
negative_examples = ["figlio"]
print(model.most_similar(positive_examples, negative_examples, top_n=1, filter=True))

[('madre', 0.56)]


In [14]:
positive_examples = ["estate", "freddo"]        # stagione fredda
negative_examples = ["caldo"]
print(model.most_similar(positive_examples, negative_examples, top_n=1, filter=True))

[('inverno', 0.5)]


In [15]:
positive_examples = ["bianca", "nero"]          # femminile di "nero"
negative_examples = ["bianco"]
print(model.most_similar(positive_examples, negative_examples, top_n=1, filter=True))

[('nera', 0.56)]


In [16]:
positive_examples = ["treno", "automobili"]     # singolare di "automobili"
negative_examples = ["treni"]
print(model.most_similar(positive_examples, negative_examples, top_n=1, filter=True))

[('automobile', 0.54)]


In [17]:
positive_examples = ["andare", "guardato"]      # infinito di "guardato"
negative_examples = ["andato"]
print(model.most_similar(positive_examples, negative_examples, top_n=1, filter=True))

[('guardare', 0.57)]


In [18]:
positive_examples = ["bello", "cattivo"]        # opposto di "cattivo"
negative_examples = ["brutto"]
print(model.most_similar(positive_examples, negative_examples, top_n=1, filter=True))

[('buono', 0.37)]


In [9]:
positive_examples = ["chitarra", "pianista"]    # strumento del pianista
negative_examples = ["chitarrista"]
print(model.most_similar(positive_examples, negative_examples, top_n=1, filter=True))

[('pianoforte', 0.63)]
